# xarray

水文水资源等地球科学方面还有一个数据处理的库是非常好用，那就是xarray。本文参考：[xarray官方文档](http://xarray.pydata.org/en/stable/)，简单记录下xarray的基本使用方法。

xarray是一个专门用来处理多维标签数据的python库。Xarray在类似于**NumPy的原始数组上以维度，坐标和属性的形式引入标签**，从而提供了更直观，更简洁和更少出错的开发人员体验。引入标签的原因是现实世界中，数据往往不是原始的多维数组，而还包含一系列反映数据编码信息的标签，比如时空信息。xarray可以利用这些标签更好地完成对数据的操作，后面会根据实际使用逐步记录。

Xarray借鉴了pandas，特别适合处理netCDF文件，netcdf是xarray数据模型的来源，如果不了解netcdf可以参考：[netcdf-python 介绍](https://github.com/OuyangWenyu/hydroGIS/blob/master/Criteria/netcdf-intro.ipynb)。xarray中最基本的数据结构是DataArray和Dataset，前者类似于pandas里的Series，后者类似DataFrame，更多信息后面记录。

另外xarray还与dask紧密集成以进行并行计算，后面用到dask再做记录。

首先，看看它的安装方法。建议conda安装：

```Shell
conda install -c conda-forge xarray
```

## 基本术语

以下无明确说明时，arr 表示DataArray对象。

- DataArray：是一个多维数组，有标签，如果name属性设置了，就成为一个named DataArray
- Dataset：一个类dict的DataArray对象集合，有多个排列的维度。Datasets有 Varaible
- Varaible：类似于netcdf的variable，包含维度，数据，属性等。variables和numpy数组的主要区别是变量上的广播运算基于维度名称的。每个DataArray有一个潜在variable可以通过arr.variable访问。Variable在Dataset和DataArray内的，所以不必单独用它。
- Dimension：一个维度轴就是固定一个维度上的所有点的集合。每个维度轴有一个名字，比如x维度。DataArray对象的维度就是被命名的维度轴。第i个维度名称可用arr.dim[i]获取。默认的维度名称是dim_0,dim_1以此类推。
- Coordinate：一个标记维度的数组。在一维情况下，坐标数组的至可以被认为是维度的标签。有两种坐标：
    - Dimension coordinate：一维的坐标数组用名字和维度名字指定给arr，可见于arr.dims。维度坐标类似于DataFrame中的index。
    - Non-dimension coordinate：非维度坐标可见于arr.coords，这些坐标数组可以是一维也可以是多维，多维情况主要见于物理坐标和逻辑坐标不一致的时候，非维度坐标是不能索引的。
- Index：index是优化数据结构用于快速索引和切片的。xarray用维度坐标可以i快速索引。

以上有些晦涩，接下来看例子。

## 基本数据结构

先看DataArray，多维标记数组。

In [1]:
import numpy as np
data = np.random.rand(4, 3)
data

array([[0.86082042, 0.1547373 , 0.28176335],
       [0.44304539, 0.39531056, 0.2847507 ],
       [0.91936809, 0.50282199, 0.42949692],
       [0.11212279, 0.18087763, 0.01919186]])

In [3]:
import pandas as pd
locs = ['IA', 'IL', 'IN']
times = pd.date_range('2000-01-01', periods=4)
times

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04'], dtype='datetime64[ns]', freq='D')

In [5]:
import xarray as xr
foo = xr.DataArray(data, coords=[times, locs], dims=['time', 'space'])
foo

<xarray.DataArray (time: 4, space: 3)>
array([[0.86082042, 0.1547373 , 0.28176335],
       [0.44304539, 0.39531056, 0.2847507 ],
       [0.91936809, 0.50282199, 0.42949692],
       [0.11212279, 0.18087763, 0.01919186]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'

可以看到这个arr有两个维度，一个时间，一个空间，时间对应一个坐标数组，是pandas的data_range，空间对应一个list，相应的数据有4\*3=12个，且数组共4行，对应4个时间，三列对应三个空间。

最简单的初始化方式是直接使用data'：

In [6]:
xr.DataArray(data)

<xarray.DataArray (dim_0: 4, dim_1: 3)>
array([[0.86082042, 0.1547373 , 0.28176335],
       [0.44304539, 0.39531056, 0.2847507 ],
       [0.91936809, 0.50282199, 0.42949692],
       [0.11212279, 0.18087763, 0.01919186]])
Dimensions without coordinates: dim_0, dim_1

可以看到默认的坐标。

另外，可以直接使用字典形式创建coords：

In [7]:
xr.DataArray(data, coords=[('time', times), ('space', locs)])

<xarray.DataArray (time: 4, space: 3)>
array([[0.86082042, 0.1547373 , 0.28176335],
       [0.44304539, 0.39531056, 0.2847507 ],
       [0.91936809, 0.50282199, 0.42949692],
       [0.11212279, 0.18087763, 0.01919186]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'

再比如多维的：

In [8]:
xr.DataArray(data, coords={'time': times, 'space': locs, 'const': 42, 
                           'ranking': (('time', 'space'), np.arange(12).reshape(4,3))},
             dims=['time', 'space'])

<xarray.DataArray (time: 4, space: 3)>
array([[0.86082042, 0.1547373 , 0.28176335],
       [0.44304539, 0.39531056, 0.2847507 ],
       [0.91936809, 0.50282199, 0.42949692],
       [0.11212279, 0.18087763, 0.01919186]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'
    const    int32 42
    ranking  (time, space) int32 0 1 2 3 4 5 6 7 8 9 10 11

如上，time和space就是维度坐标，const和ranking就是非维度坐标。

另外，还可以使用DataFrame来初始化：

In [9]:
df = pd.DataFrame({'x': [0, 1], 'y': [2, 3]}, index=['a', 'b'])
df

,x,y
a,0,2
b,1,3


In [10]:
df.index.name = 'abc'
df

,x,y
abc,,
a,0,2
b,1,3


In [11]:
df.columns.name = 'xyz'
df

xyz,x,y
abc,,
a,0,2
b,1,3


In [12]:
xr.DataArray(df)

<xarray.DataArray (abc: 2, xyz: 2)>
array([[0, 2],
       [1, 3]], dtype=int64)
Coordinates:
  * abc      (abc) object 'a' 'b'
  * xyz      (xyz) object 'x' 'y'

接下来看看数组的属性：

In [13]:
foo.values

array([[0.86082042, 0.1547373 , 0.28176335],
       [0.44304539, 0.39531056, 0.2847507 ],
       [0.91936809, 0.50282199, 0.42949692],
       [0.11212279, 0.18087763, 0.01919186]])

注意DataArray中的数组值都是统一的数据类型。如果需要不同数据类型的，那么需要使用Dataset。

In [14]:
foo.dims

('time', 'space')

In [15]:
foo.coords

Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'

In [16]:
foo.attrs

{}

In [17]:
print(foo.name)

None


以上有缺失默认值的，可以使用下列方式补充：

In [18]:
foo.name = 'foo'
foo.attrs['units'] = 'meters'
foo

<xarray.DataArray 'foo' (time: 4, space: 3)>
array([[0.86082042, 0.1547373 , 0.28176335],
       [0.44304539, 0.39531056, 0.2847507 ],
       [0.91936809, 0.50282199, 0.42949692],
       [0.11212279, 0.18087763, 0.01919186]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'
Attributes:
    units:    meters

使用rename会返回一个新的数据数组：

In [19]:
foo.rename('bar')

<xarray.DataArray 'bar' (time: 4, space: 3)>
array([[0.86082042, 0.1547373 , 0.28176335],
       [0.44304539, 0.39531056, 0.2847507 ],
       [0.91936809, 0.50282199, 0.42949692],
       [0.11212279, 0.18087763, 0.01919186]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'
Attributes:
    units:    meters

坐标是类似dict类型的

In [20]:
foo.coords['time']

<xarray.DataArray 'time' (time: 4)>
array(['2000-01-01T00:00:00.000000000', '2000-01-02T00:00:00.000000000',
       '2000-01-03T00:00:00.000000000', '2000-01-04T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04

In [21]:
foo['time']

<xarray.DataArray 'time' (time: 4)>
array(['2000-01-01T00:00:00.000000000', '2000-01-02T00:00:00.000000000',
       '2000-01-03T00:00:00.000000000', '2000-01-04T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04

坐标可以被删除：

In [22]:
foo['ranking'] = ('space', [1, 2, 3])
foo.coords

Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'
    ranking  (space) int32 1 2 3

In [23]:
del foo['ranking']
foo.coords

Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'

下面看Dataset，它就是为netcdf定制的。

In [24]:
temp = 15 + 8 * np.random.randn(2, 2, 3)
temp

array([[[ 9.38784084,  1.32272273, 11.10376   ],
        [13.02591895, 17.73363208, 18.01805839]],

       [[11.63603795, 19.42711602, 10.41797777],
        [-6.07555963, 10.90500841, 20.84515736]]])

In [25]:
precip = 10 * np.random.rand(2, 2, 3)
precip 

array([[[6.27147192, 3.32837045, 8.85742054],
        [4.35676988, 3.63088735, 0.09502339]],

       [[7.17492778, 0.11402965, 1.72007276],
        [1.99444539, 0.1965453 , 1.92094201]]])

In [26]:
lon = [[-99.83, -99.32], [-99.79, -99.23]]
lat = [[42.25, 42.21], [42.63, 42.59]]
ds = xr.Dataset({'temperature': (['x', 'y', 'time'],  temp),
                     'precipitation': (['x', 'y', 'time'], precip)},
                    coords={'lon': (['x', 'y'], lon),
                            'lat': (['x', 'y'], lat),
                            'time': pd.date_range('2014-09-06', periods=3),
                            'reference_time': pd.Timestamp('2014-09-05')})
ds

<xarray.Dataset>
Dimensions:         (time: 3, x: 2, y: 2)
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y
Data variables:
    temperature     (x, y, time) float64 9.388 1.323 11.1 ... -6.076 10.91 20.85
    precipitation   (x, y, time) float64 6.271 3.328 8.857 ... 0.1965 1.921

可以传入arr或者pandas到Dataset：

In [27]:
xr.Dataset({'bar': foo})

<xarray.Dataset>
Dimensions:  (space: 3, time: 4)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'IL' 'IN'
Data variables:
    bar      (time, space) float64 0.8608 0.1547 0.2818 ... 0.1809 0.01919

In [28]:
xr.Dataset({'bar': foo.to_pandas()})

<xarray.Dataset>
Dimensions:  (space: 3, time: 4)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) object 'IA' 'IL' 'IN'
Data variables:
    bar      (time, space) float64 0.8608 0.1547 0.2818 ... 0.1809 0.01919

这里就可以理解variable的意义了。

接下来看看Dataset的各个属性：

In [29]:
'temperature' in ds

True

In [30]:
ds['temperature']

<xarray.DataArray 'temperature' (x: 2, y: 2, time: 3)>
array([[[ 9.38784084,  1.32272273, 11.10376   ],
        [13.02591895, 17.73363208, 18.01805839]],

       [[11.63603795, 19.42711602, 10.41797777],
        [-6.07555963, 10.90500841, 20.84515736]]])
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y

In [31]:
ds.data_vars

Data variables:
    temperature    (x, y, time) float64 9.388 1.323 11.1 ... -6.076 10.91 20.85
    precipitation  (x, y, time) float64 6.271 3.328 8.857 ... 1.994 0.1965 1.921

In [32]:
ds.coords

Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05

In [33]:
ds.attrs

{}

In [34]:
ds.attrs['title'] = 'example attribute'
ds

<xarray.Dataset>
Dimensions:         (time: 3, x: 2, y: 2)
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y
Data variables:
    temperature     (x, y, time) float64 9.388 1.323 11.1 ... -6.076 10.91 20.85
    precipitation   (x, y, time) float64 6.271 3.328 8.857 ... 0.1965 1.921
Attributes:
    title:    example attribute

In [35]:
ds.temperature

<xarray.DataArray 'temperature' (x: 2, y: 2, time: 3)>
array([[[ 9.38784084,  1.32272273, 11.10376   ],
        [13.02591895, 17.73363208, 18.01805839]],

       [[11.63603795, 19.42711602, 10.41797777],
        [-6.07555963, 10.90500841, 20.84515736]]])
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y